In [1]:
import torch
import torch.nn as nn

# Сеть с множеством выходов 

Если сеть имеет множество выходов, то backward по их сумме эквивалентен backward-у по каждой из них 

## Пример 

In [2]:
criterion = nn.CrossEntropyLoss()

In [3]:
def get_losses():
    torch.manual_seed(89)
    input = torch.randn(3, 5, requires_grad= True)

    target = torch.empty(3, dtype = torch.long).random_(5)
    loss1 = criterion(input, target) 

    target = torch.empty(3, dtype = torch.long).random_(5)
    loss2 = criterion(input, target)

    target = torch.empty(3, dtype = torch.long).random_(5)
    loss3 = criterion(input, target)
    return input, loss1, loss2, loss3

### backward по каждому выходу

In [5]:
input, loss1, loss2, loss3 = get_losses()

In [6]:
loss1.backward(retain_graph=True)

In [7]:
loss2.backward(retain_graph=True)

In [8]:
loss3.backward()

In [9]:
print(input.grad)

tensor([[-0.1326,  0.1283,  0.1131,  0.1621, -0.2709],
        [ 0.4744, -0.5847,  0.2551, -0.3049,  0.1600],
        [-0.0211,  0.0576, -0.1788,  0.0728,  0.0695]])


### backward по сумме ошибок

In [10]:
input, loss1, loss2, loss3 = get_losses()

In [11]:
loss = loss1+loss2+loss3
loss.backward()
print(input.grad)

tensor([[-0.1326,  0.1283,  0.1131,  0.1621, -0.2709],
        [ 0.4744, -0.5847,  0.2551, -0.3049,  0.1600],
        [-0.0211,  0.0576, -0.1788,  0.0728,  0.0695]])
